In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')
import emoji
import contractions
import re
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
#Here we read the data, drop columns we don't use, and drop reviews longer than 150 words.
dataset=pd.read_csv('stctv_reviews.csv')
dataset

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9d7ecf76-ee66-41ac-a6dd-9de8902d49af,MOHAMMED ALKHAMIS,https://play-lh.googleusercontent.com/a-/AD_cM...,It needs improvement. Very slow in loading up....,2,0,6.1.0,2023-06-12 07:13:07,NaN,NaN,6.1.0
1,2d4f22b1-c39d-4721-8239-96a24553af9c,ya or,https://play-lh.googleusercontent.com/a-/AD_cM...,لايعمل مع سامسونج A72,1,0,6.1.0,2023-06-10 03:53:21,NaN,NaN,6.1.0
2,270fc5c7-1b25-4bcd-991f-35189a470d31,Alamin Molla Molla,https://play-lh.googleusercontent.com/a-/AD_cM...,Very nice.Thanks for stc tv,5,0,6.1.0,2023-06-09 10:59:19,NaN,NaN,6.1.0
3,9cb85a0a-b7b5-4758-ad76-4d5868e79ed4,OnelNeth TV,https://play-lh.googleusercontent.com/a-/AD_cM...,great off or online streaming app.,5,0,6.1.0,2023-06-06 01:01:03,NaN,NaN,6.1.0
4,d6dc9dfd-2966-4843-84dc-06039f9f1692,أيمنAiman53-53,https://play-lh.googleusercontent.com/a-/AD_cM...,"A great application with variety, but I can't ...",3,2,6.1.0,2023-06-05 12:06:35,NaN,NaN,6.1.0
...,...,...,...,...,...,...,...,...,...,...,...
1374,54904ccd-cefc-4ee5-9295-2cbaec0ad533,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,جوّي على جوّي⁦❤️,5,1,NaN,2018-04-16 10:06:23,NaN,NaN,NaN
1375,f6343528-b344-4c9e-98fa-71a47540d7d5,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's good network,5,1,2.3.2,2018-04-08 00:17:12,NaN,NaN,2.3.2
1376,94a8d60f-d594-473f-b32b-90deddcb1e75,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"It just says that I'm using another screen, bu...",1,2,2.3.2,2018-03-27 04:43:16,NaN,NaN,2.3.2
1377,e32fc915-6782-4e35-819f-94e52272f898,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This app not support Android TV.,3,1,NaN,2018-03-26 07:44:27,NaN,NaN,NaN


In [3]:
data_in = dataset.copy()
data_in = data_in.drop_duplicates(subset="content")  # Drop dupes in Reviews
data_in.drop(["reviewId", "userName",
             "userImage","score","thumbsUpCount","reviewCreatedVersion","replyContent","repliedAt"], axis=1, inplace=True)
data_in["Length"] = data_in["content"].str.split(" ").str.len()
data_in = data_in[data_in.Length < 150]
data_in.drop(["Length"], axis=1, inplace=True)
data = data_in
data

,content,at,appVersion
0,It needs improvement. Very slow in loading up....,2023-06-12 07:13:07,6.1.0
1,لايعمل مع سامسونج A72,2023-06-10 03:53:21,6.1.0
2,Very nice.Thanks for stc tv,2023-06-09 10:59:19,6.1.0
3,great off or online streaming app.,2023-06-06 01:01:03,6.1.0
4,"A great application with variety, but I can't ...",2023-06-05 12:06:35,6.1.0
...,...,...,...
1374,جوّي على جوّي⁦❤️,2018-04-16 10:06:23,NaN
1375,It's good network,2018-04-08 00:17:12,2.3.2
1376,"It just says that I'm using another screen, bu...",2018-03-27 04:43:16,2.3.2
1377,This app not support Android TV.,2018-03-26 07:44:27,NaN


In [4]:
def clean_text(text):
    text = emoji.demojize(text)  # fix emojis
    text = contractions.fix(text)  # fix contractions
    text = re.sub(r'[^\x00-\x7f]', r'', text)  # remove strange fonts
    text = re.sub(r"\d+", "number", text)  # replace numbers with "number"
    text = re.sub(r'[^\w\s]', '', text)  # remove non-alphanumeric chars
    text = text.replace('_', ' ')  # replace underscores with space
    text = text.strip()  # strip extra spaces
    return text


In [5]:
data["content"] = data["content"].apply(clean_text)
data

,content,at,appVersion
0,It needs improvement Very slow in loading up T...,2023-06-12 07:13:07,6.1.0
1,Anumber,2023-06-10 03:53:21,6.1.0
2,Very niceThanks for stc tv,2023-06-09 10:59:19,6.1.0
3,great off or online streaming app,2023-06-06 01:01:03,6.1.0
4,A great application with variety but I cannot ...,2023-06-05 12:06:35,6.1.0
...,...,...,...
1374,red heart,2018-04-16 10:06:23,NaN
1375,It is good network,2018-04-08 00:17:12,2.3.2
1376,It just says that I am using another screen bu...,2018-03-27 04:43:16,2.3.2
1377,This app not support Android TV,2018-03-26 07:44:27,NaN


In [6]:
#Checking for null values
data.isnull().sum()

content         0
at              0
appVersion    219
dtype: int64

In [7]:
!pip install textstat
!pip install nltk


In [8]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from textstat import flesch_reading_ease

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [9]:
sid = SentimentIntensityAnalyzer()
words = set(nltk.corpus.words.words())


In [10]:
positive_scores = []
negative_scores = []
neutral_scores = []
compound_scores = []

In [11]:
for index, row in data.iterrows():
    contents = row['content']
    
    sentiment_scores = sid.polarity_scores(contents)
    
    positive_score = sentiment_scores['pos']
    negative_score = sentiment_scores['neg']
    neutral_score = sentiment_scores['neu']
    compound_score = sentiment_scores['compound']
    
    positive_scores.append(positive_score * 100)
    negative_scores.append(negative_score * 100)
    neutral_scores.append(neutral_score * 100)
    compound_scores.append(compound_score * 100)

In [12]:
data['POSITIVE SCORE'] = positive_scores
data['NEGATIVE SCORE'] = negative_scores
data['NEUTRAL SCORE'] = neutral_scores
data['COMPOUND SCORE'] = compound_scores


data

,content,at,appVersion,POSITIVE SCORE,NEGATIVE SCORE,NEUTRAL SCORE,COMPOUND SCORE
0,It needs improvement Very slow in loading up T...,2023-06-12 07:13:07,6.1.0,33.6,5.0,61.4,82.28
1,Anumber,2023-06-10 03:53:21,6.1.0,0.0,0.0,100.0,0.00
2,Very niceThanks for stc tv,2023-06-09 10:59:19,6.1.0,0.0,0.0,100.0,0.00
3,great off or online streaming app,2023-06-06 01:01:03,6.1.0,45.1,0.0,54.9,62.49
4,A great application with variety but I cannot ...,2023-06-05 12:06:35,6.1.0,22.1,0.0,77.9,37.16
...,...,...,...,...,...,...,...
1374,red heart,2018-04-16 10:06:23,NaN,0.0,0.0,100.0,0.00
1375,It is good network,2018-04-08 00:17:12,2.3.2,49.2,0.0,50.8,44.04
1376,It just says that I am using another screen bu...,2018-03-27 04:43:16,2.3.2,18.6,15.2,66.1,16.95
1377,This app not support Android TV,2018-03-26 07:44:27,NaN,0.0,31.1,68.9,-30.89


In [13]:
positive_threshold = 0.2
negative_threshold = -0.2

data['Sentiment']= ''

for index, row in data.iterrows():
    compound_score = row['COMPOUND SCORE']
    
    if compound_score > positive_threshold:
        data.at[index, 'Sentiment'] = 'Positive'
    elif compound_score < negative_threshold:
            data.at[index, 'Sentiment'] = 'Negative'
    else:
                data.at[index, 'Sentiment'] = 'Neutral'

In [14]:
data.head(50)

,content,at,appVersion,POSITIVE SCORE,NEGATIVE SCORE,NEUTRAL SCORE,COMPOUND SCORE,Sentiment
0,It needs improvement Very slow in loading up T...,2023-06-12 07:13:07,6.1.0,33.6,5.0,61.4,82.28,Positive
1,Anumber,2023-06-10 03:53:21,6.1.0,0.0,0.0,100.0,0.00,Neutral
2,Very niceThanks for stc tv,2023-06-09 10:59:19,6.1.0,0.0,0.0,100.0,0.00,Neutral
3,great off or online streaming app,2023-06-06 01:01:03,6.1.0,45.1,0.0,54.9,62.49,Positive
4,A great application with variety but I cannot ...,2023-06-05 12:06:35,6.1.0,22.1,0.0,77.9,37.16,Positive
5,Application getting hanged aloteven after clea...,2023-06-03 08:00:37,6.1.0,13.2,0.0,86.8,49.39,Positive
6,Unfortunately the app is not available on Goog...,2023-06-02 14:46:11,6.1.0,27.6,13.0,59.5,40.19,Positive
7,excellent,2023-06-02 10:19:06,6.1.0,100.0,0.0,0.0,57.19,Positive
8,it is great app,2023-06-01 05:38:33,6.1.0,57.7,0.0,42.3,62.49,Positive
9,it is ok,2023-06-01 01:22:30,6.1.0,52.4,0.0,47.6,29.60,Positive


In [15]:
data['Sentiment'].value_counts()

Neutral     513
Positive    439
Negative    225
Name: Sentiment, dtype: int64

In [16]:
data.to_csv('SentimentSTCTV.csv', index=False)